In [22]:
import pandas as pd
import spacy
nlp = spacy.load("ru_core_news_lg")
import RAKE
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('russian')
STOPWORDS.extend(['который'])
import summa
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from typing import List, Iterable, Dict

In [23]:
def preprocess_text(text):
    doc = nlp(text)
    lemmata = [token.lemma_ for token in doc]
    lemmata = ['который' if l.startswith('котор') else l for l in lemmata]
    return ' '.join(lemmata)

В качестве текстов были взяты новости с сайта [hse.ru](https://www.hse.ru/news/). В них уже выделены ключевые слова и/или тэги.

Ключевые слова с сайта даны в колонке `original_keywords`, выделенные вручную -- в колонке `manual_keywords`. Количество пересечений в этих двух наборах ключевых слов -- в колонке `intersection`.

In [24]:
corpus = pd.read_csv('keywords_corpus.csv', sep=';')
corpus['lemmatized_text'] = corpus['text'].apply(preprocess_text)
corpus['original_keywords'] = corpus['original_keywords'].apply(lambda x: set(x.split(', ')))
corpus['manual_keywords'] = corpus['manual_keywords'].apply(lambda x: set(x.lower().split(', ')))
corpus['intersection'] = corpus.apply(lambda x: len(x['original_keywords'].intersection(x['manual_keywords'])), axis=1)
corpus

,text,original_keywords,manual_keywords,lemmatized_text,intersection
0,Завершились региональные кейс-чемпионаты школь...,{кейс-чемпионат школьников по экономике и пред...,"{кейс-чемпионат школьников, предпринимательств...",завершиться региональный кейс - чемпионаты шко...,0
1,Премьер-министр Михаил Мишустин предложил Ярос...,"{новое в ВШЭ, официально, экспертиза, идеи и о...","{ярослав кузьминов, экспертиза, высшая школа э...",премьер - министр михаил мишустин предложить я...,1
2,Сегодня публикацией предметного рейтинга «Иску...,"{официально, магистратура, рейтинг QS, рейтинг...","{предмет, академический рейтинг мировых универ...","сегодня публикация предметный рейтинг "" искусс...",0
3,Компания Яндекс создала консультативный совет ...,"{профессора, экспертиза, идеи и опыт}","{консультативный совет, ярослав кузьминов, экс...",компания яндекс создать консультативный совет ...,1
4,"Абитуриентов ждет увеличение бюджетных мест, р...","{новое в ВШЭ, официально, магистратура, разъяс...","{вступительные испытания, магистратура, обучен...","абитуриент ждать увеличение бюджетный место , ...",2
5,До 5 ноября продолжается прием на совместную п...,"{приглашение к участию, дополнительное образов...","{спортивный, спортивный менеджмент, проект, сп...",до 5 ноябрь продолжаться приём на совместный п...,1
6,29-30 октября в онлайн-формате пройдет юбилейн...,"{международное сотрудничество, международная к...","{международное сотрудничество, финансовый, раб...",29 - 30 октябрь в онлайн - формате пройти юбил...,2
7,НИУ ВШЭ вручена ежегодная премия «Импульс добр...,"{репортаж о событии, достижения, социальное пр...","{фонд «наше будущее», социальный, «импульс доб...","ниу вшэ вручить ежегодный премия "" импульс доб...",1


Создадим эталон разметки, объединив ключевые слова, размеченные вручную, и ключевые слова с сайта, предварительно удалив из них те, что не встречаются в тексте. Эталон разметки находится в колонке `gold_keywords`.

In [25]:
corpus['original_keywords'] = corpus.apply(lambda x: [i for i in x['original_keywords'] if i in x['lemmatized_text']], axis=1)
corpus['original_keywords'] = corpus['original_keywords'].apply(set)
corpus['gold_keywords'] = corpus.apply(lambda x: x['original_keywords'].union(x['manual_keywords']), axis=1)
corpus

,text,original_keywords,manual_keywords,lemmatized_text,intersection,gold_keywords
0,Завершились региональные кейс-чемпионаты школь...,{},"{кейс-чемпионат школьников, предпринимательств...",завершиться региональный кейс - чемпионаты шко...,0,"{кейс-чемпионат школьников, предпринимательств..."
1,Премьер-министр Михаил Мишустин предложил Ярос...,{экспертиза},"{ярослав кузьминов, экспертиза, высшая школа э...",премьер - министр михаил мишустин предложить я...,1,"{ярослав кузьминов, экспертное сообщество, экс..."
2,Сегодня публикацией предметного рейтинга «Иску...,{},"{предмет, академический рейтинг мировых универ...","сегодня публикация предметный рейтинг "" искусс...",0,"{предмет, академический рейтинг мировых универ..."
3,Компания Яндекс создала консультативный совет ...,{},"{консультативный совет, ярослав кузьминов, экс...",компания яндекс создать консультативный совет ...,1,"{консультативный совет, ярослав кузьминов, экс..."
4,"Абитуриентов ждет увеличение бюджетных мест, р...","{бакалавриат, магистратура}","{вступительные испытания, магистратура, обучен...","абитуриент ждать увеличение бюджетный место , ...",2,"{вступительные испытания, программа, бакалаври..."
5,До 5 ноября продолжается прием на совместную п...,{},"{спортивный, спортивный менеджмент, проект, сп...",до 5 ноябрь продолжаться приём на совместный п...,1,"{спортивный, спортивный менеджмент, проект, сп..."
6,29-30 октября в онлайн-формате пройдет юбилейн...,{},"{международное сотрудничество, финансовый, раб...",29 - 30 октябрь в онлайн - формате пройти юбил...,2,"{международное сотрудничество, финансовый, раб..."
7,НИУ ВШЭ вручена ежегодная премия «Импульс добр...,{},"{фонд «наше будущее», социальный, «импульс доб...","ниу вшэ вручить ежегодный премия "" импульс доб...",1,"{фонд «наше будущее», социальный, «импульс доб..."


### RAKE

In [26]:
rake = RAKE.Rake(STOPWORDS)

In [27]:
corpus['rake_keywords'] = corpus['lemmatized_text'].apply(lambda x: [i[0] for i in rake.run(x, maxWords=3, minFrequency=2)])

### TextRank

In [28]:
corpus['textrank_keywords'] = corpus['lemmatized_text'].apply(lambda x: summa.keywords.keywords(x, language='russian', 
                                                                                                additional_stopwords=STOPWORDS
                                                                                               ).split('\n'))

### TF-IDF

In [29]:
vect = TfidfVectorizer(stop_words=STOPWORDS)

In [30]:
kw_idx = pd.Series(list(np.argsort(-vect.fit_transform(corpus['lemmatized_text']).toarray())))
corpus['tfidf_keywords'] = kw_idx.apply(lambda x: vect.get_feature_names_out()[x][:20])

In [31]:
corpus

,text,original_keywords,manual_keywords,lemmatized_text,intersection,gold_keywords,rake_keywords,textrank_keywords,tfidf_keywords
0,Завершились региональные кейс-чемпионаты школь...,{},"{кейс-чемпионат школьников, предпринимательств...",завершиться региональный кейс - чемпионаты шко...,0,"{кейс-чемпионат школьников, предпринимательств...","[президент московский торгово, региональный ке...","[команда, бизнес, чемпионаты, чемпионат, регио...","[кейс, чемпионат, победитель, палата, московск..."
1,Премьер-министр Михаил Мишустин предложил Ярос...,{экспертиза},"{ярослав кузьминов, экспертиза, высшая школа э...",премьер - министр михаил мишустин предложить я...,1,"{ярослав кузьминов, экспертное сообщество, экс...","[ведущий вуз страна, учитывать ваш опыт, экспе...","[ярослав, правительство, это, работа, новый, д...","[экспертный, ярослав, министр, правительство, ..."
2,Сегодня публикацией предметного рейтинга «Иску...,{},"{предмет, академический рейтинг мировых универ...","сегодня публикация предметный рейтинг "" искусс...",0,"{предмет, академический рейтинг мировых универ...","[мировой рейтинг, гуманитарный наука, год, qs,...","[хороший, вшэ, предметный рейтинг, год высокий...","[рейтинг, предметный, топ, qs, год, университе..."
3,Компания Яндекс создала консультативный совет ...,{},"{консультативный совет, ярослав кузьминов, экс...",компания яндекс создать консультативный совет ...,1,"{консультативный совет, ярослав кузьминов, экс...","[игорь агамирзян, экономический институт, проф...","[экосистема, экосистемы, яндекс, цифровой, кон...","[экосистема, яндекс, совет, консультативный, ц..."
4,"Абитуриентов ждет увеличение бюджетных мест, р...","{бакалавриат, магистратура}","{вступительные испытания, магистратура, обучен...","абитуриент ждать увеличение бюджетный место , ...",2,"{вступительные испытания, программа, бакалаври...","[управление стратегический коммуникация, между...","[программа, программы, программ, программам, п...","[бакалавриат, абитуриент, программа, приём, мо..."
5,До 5 ноября продолжается прием на совместную п...,{},"{спортивный, спортивный менеджмент, проект, сп...",до 5 ноябрь продолжаться приём на совместный п...,1,"{спортивный, спортивный менеджмент, проект, сп...","[высокий школа экономика, спортивный менеджмен...","[продолжать, ноябрь продолжаться приём, готовы...","[спортивный, программа, менеджмент, знание, се..."
6,29-30 октября в онлайн-формате пройдет юбилейн...,{},"{международное сотрудничество, финансовый, раб...",29 - 30 октябрь в онлайн - формате пройти юбил...,2,"{международное сотрудничество, финансовый, раб...","[финансовый экономика, международный уровень, ...","[конференция, работа, участник, стать, статья,...","[конференция, финансовый, участник, миэф, учён..."
7,НИУ ВШЭ вручена ежегодная премия «Импульс добр...,{},"{фонд «наше будущее», социальный, «импульс доб...","ниу вшэ вручить ежегодный премия "" импульс доб...",1,"{фонд «наше будущее», социальный, «импульс доб...","[программа профессиональный переподготовка, сф...","[свой, социальный предпринимательство, обществ...","[социальный, премия, наш, предпринимательство,..."


## Фильтрация

По-моему, в качестве ключевых слов плохо подходят глаголы, а также сочетания больше двух существительных. Отфильтруем их:

In [32]:
def filter_keywords(kws: List[str]) -> List[str]:
    filtered_kws = []
    for kw in kws:
        kw_doc = nlp(kw)
        noun_count = 0
        for token in kw_doc:
            if token.pos_ == 'VERB':
                break
            elif token.pos_ == 'NOUN':
                noun_count += 1
            if noun_count >= 3:
                break
        else:
            filtered_kws.append(kw)
    return filtered_kws

## Метрики

In [33]:
def compute_metrics(gold_kws: Iterable[Iterable[str]], pred_kws: Iterable[Iterable[str]]) -> Dict:
    n_samples = len(gold_kws)
    precision_avg = []
    recall_avg = []
    f1_avg = []
    for g, p in zip(gold_kws, pred_kws):
        g = set(g)
        p = set(p)
        tp = len(g.intersection(p))
        if tp == 0:
            precision = 0
            recall = 0
            f1 = 0
        else:
            precision = tp / len(p)
            recall = tp / len(g)
            f1 = 2 * precision * recall / (precision + recall)
        precision_avg.append(precision)
        recall_avg.append(recall)
        f1_avg.append(f1)
    precision_avg = sum(precision_avg) / n_samples
    recall_avg = sum(recall_avg) / n_samples
    f1_avg = sum(f1_avg) / n_samples
    return {'precision': precision_avg, 'recall': recall_avg, 'f1_score': f1_avg}

In [34]:
methods = ['rake', 'textrank', 'tfidf']

### Без фильтра

In [35]:
for m in methods:
    metrics = compute_metrics(corpus['gold_keywords'], corpus[f'{m}_keywords'])
    print(m)
    print(metrics)

rake
{'precision': 0.08735225927070059, 'recall': 0.14346070596070595, 'f1_score': 0.10173615390936357}
textrank
{'precision': 0.06875320935103543, 'recall': 0.28146853146853146, 'f1_score': 0.10891194120603467}
tfidf
{'precision': 0.21874999999999997, 'recall': 0.36401098901098905, 'f1_score': 0.27207210067148396}


### С фильтром

In [36]:
for m in methods:
    metrics = compute_metrics(corpus['gold_keywords'].apply(filter_keywords), corpus[f'{m}_keywords'].apply(filter_keywords))
    print(m)
    print(metrics)

rake
{'precision': 0.09607546311566267, 'recall': 0.15802947052947056, 'f1_score': 0.11385787701577177}
textrank
{'precision': 0.08989398711920264, 'recall': 0.3066433566433566, 'f1_score': 0.13562403807676918}
tfidf
{'precision': 0.23028250773993808, 'recall': 0.4053613053613054, 'f1_score': 0.29120519821298463}


## Ошибки методов выделения ключевых слов

Добавление морфологических фильтров позволяет немного повысить метрики.

В ключевые слова, выделенные TextRank, могут попадать разные формы одного и того же слова:  

In [19]:
corpus['textrank_keywords'][4][:10]

['программа',
 'программы',
 'программ',
 'программам',
 'программу',
 'москва',
 'год',
 'приём',
 'прием',
 'свой']

Это можно решить более точной лемматизацией.

В ключевых словах Rake встречаются длинные специфические словосочетания вроде *учитывать ваш опыт*:

In [40]:
corpus['rake_keywords'][1]

['ведущий вуз страна',
 'учитывать ваш опыт',
 'экспертный совет',
 'экспертный сообщество',
 'это',
 'правительство',
 'проект',
 'министр',
 'экономика',
 'премьер',
 'подчеркнуть',
 'число',
 'хотеть']

Возможно, это можно решить, подобрав параметры модели Rake.

В целом, методы плохо выделяют частотные слова, которые, тем не менее, несут смысл и являются рамочными для текста (в данном корпусе это слово *университет* и подобные).